##Step 0. 환경준비/ 라이브러리 설치

In [34]:
# !pip install datasets
# !pip install loralib
# !pip install trl
# !pip install accelerate
# !pip install transformers
# !pip install evaluate
#!pip install sacrebleu rouge-score
# #실행 후 런타임 재시작

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.9 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=59a3320fbe573d8453d191a006f1a4d287e93e74c9b5c6a84f47e21a70fdde50
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


##Step 0. 환경 설정 / 라이브러리 임포트

In [2]:
!git clone https://github.com/airobotlab/KoChatGPT
!cp -r KoChatGPT/colossalai_ChatGPT_230319/chatgpt chatgpt

import os

modifications = [
    {
        "file": "chatgpt/trainer/callbacks/save_checkpoint.py",
        "changes": [
            {"line": 3, "old": "from chatgpt.trainer.strategies import ColossalAIStrategy, Strategy",
             "new": "from chatgpt.trainer.strategies import Strategy"},
            {"line": 71, "old": "only_rank0 = not isinstance(self.strategy, ColossalAIStrategy)",
             "new": "            only_rank0 = not isinstance(self.strategy)"},
        ],
    },
    {
        "file": "chatgpt/trainer/strategies/__init__.py",
        "changes": [
            {"line": 1, "old": "from .colossalai import ColossalAIStrategy", "new": ""},  # 삭제
            {"line": 5, "old": "__all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy', 'ColossalAIStrategy']",
             "new": "__all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy']"},
        ],
    },
    {
        "file": "chatgpt/dataset/reward_dataset.py",
        "changes": [
            {"line": 3, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ],
    },
    {
        "file": "chatgpt/trainer/base.py",
        "changes": [
            {"line": 8, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ]
    },
    {
        "file": "chatgpt/trainer/rm.py",
        "changes": [
            {"line": 8, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ]
    }
]

#파일에서 지정된 줄을 찾아 내용을 수정
def modify_file(file_path, changes):


    if not os.path.exists(file_path):
        print(f"⚠️ 파일이 존재하지 않습니다: {file_path}")
        return

    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    modified = False

    for change in changes:
        line_index = change["line"]
        if 0 <= line_index < len(lines):
            if lines[line_index].strip() == change["old"]:
                lines[line_index] = change["new"] + "\n"
                modified = True
            else:
                print(f"⚠️ {file_path} 파일의 {change['line']}번째 줄이 예상과 다릅니다.")
                print(f"   예상: {change['old']}")
                print(f"   실제: {lines[line_index].strip()}")

    if modified:
        with open(file_path, "w", encoding="utf-8") as file:
            file.writelines(lines)
        print(f"✅ 수정 완료: {file_path}")
    else:
        print(f"⚠️ {file_path} 수정할 내용이 없습니다.")

for mod in modifications:
    modify_file(mod["file"], mod["changes"])

fatal: destination path 'KoChatGPT' already exists and is not an empty directory.
⚠️ chatgpt/trainer/callbacks/save_checkpoint.py 파일의 3번째 줄이 예상과 다릅니다.
   예상: from chatgpt.trainer.strategies import ColossalAIStrategy, Strategy
   실제: from chatgpt.trainer.strategies import Strategy
⚠️ chatgpt/trainer/callbacks/save_checkpoint.py 파일의 71번째 줄이 예상과 다릅니다.
   예상: only_rank0 = not isinstance(self.strategy, ColossalAIStrategy)
   실제: only_rank0 = not isinstance(self.strategy)
⚠️ chatgpt/trainer/callbacks/save_checkpoint.py 수정할 내용이 없습니다.
⚠️ chatgpt/trainer/strategies/__init__.py 파일의 1번째 줄이 예상과 다릅니다.
   예상: from .colossalai import ColossalAIStrategy
   실제: 
⚠️ chatgpt/trainer/strategies/__init__.py 파일의 5번째 줄이 예상과 다릅니다.
   예상: __all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy', 'ColossalAIStrategy']
   실제: __all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy']
⚠️ chatgpt/trainer/strategies/__init__.py 수정할 내용이 없습니다.
⚠️ chatgpt/dataset/reward_dataset.py 파일의 3번째 줄이 예상과 다릅니다.
   예상: from tqdm import

##라이브러리 버전 확인 및 dirve 연결
학습에 필요한 autotokenizer, autoModelForCauseLM등을 import

추후 원활한 수행을 위해 drive와 연결

In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy

print("Torch version:{}".format(torch.__version__)) # Torch version:1.12.1
print("Cuda version: {}".format(torch.version.cuda)) # Cuda version: 11.3
print("transformers version: {}".format(transformers.__version__)) # transformers 4.28.0
print("GPU 사용 가능여부: {}".format(torch.cuda.is_available()))

from google.colab import drive
drive.mount('/content/drive')

# 만일 아래 모듈이 불러와지지 않는다면 Clone 및 수정을 잘 진행했는지 확인해주세요.
from chatgpt.trainer.strategies import NaiveStrategy

Torch version:2.8.0+cu126
Cuda version: 12.6
transformers version: 4.56.1
GPU 사용 가능여부: True
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

###Step 1. 데이터 및 디코딩 성능 확인

In [5]:
input_txt =  "바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."

tokens = tokenizer(input_txt).tokens()
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].numpy()

pd.options.display.max_columns = 40
pd.options.display.max_rows = 60
df = pd.DataFrame([tokens, input_ids[0]], index=["kogpt-2_tokens", "Input_IDs"])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
kogpt-2_tokens,▁바람,도,▁없는,▁공중에,▁수직,의,▁파,문을,▁내,이며,▁고,요,히,▁떨어지는,▁오동,잎은,▁누,구의,▁발자,취,▁입,니까,.
Input_IDs,10891,7235,9712,49207,14438,8143,9203,9941,9094,9639,9065,8084,8811,21215,34769,19985,9669,10139,21626,8408,9241,23775,389


In [7]:
# 디코딩 성능 확인
max_length=128
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇다면 그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리


처음 생성한 답변을 반복적으로 수행하는 모습이 보인다.
앞서 생성한 토큰의 영향을 많이 받음

In [8]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, top_p=0.90)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."
"그렇지 않습니다."
그녀는 다시 한 번 고개를 끄덕였다.
"아니오. 그건 그렇다 치지만."
"무슨 말씀이신가요?"
이번에도 그녀는 고개를 갸웃거렸다.
"물론 아닙니다. 저는 그렇게 생각합니다만."
그녀의 입가에 미소가 피어올랐다.
"저도 그렇습니다. 저도 그런 생각을 하고 있습니다."
그러자 그녀의 입에서 웃음이 새어 나왔다.
"이봐, 이봐. 괜찮아.


단순히 디코딩에서 beam search를 적용해주는것만으로 어느정도의 품질은 기대할 수 있지만
모델 자체의 성능을 끌어올린다면 더 경제적으로 고품질의 출력을 만들 수 있을것이라 생각함.


###Step 2. Supervised Fine-Tuning

베이스 모델 : skt/kogpt2-base-v2

데이터 : {prompt, completion} 데이터쌍




In [9]:
from typing import Optional, Dict, Sequence
from torch.utils.data import Dataset
from dataclasses import dataclass
import logging
import copy
import json

In [10]:
# 모델, 토크나이저 호출

model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<usr>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<sys>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=

In [11]:
# 모델 인퍼런스 단계에서 사용할 prompt 딕셔너리 템플릿과 SFT 데이터셋 클래스를 정의

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [12]:
@dataclass
class DataCollatorForSupervisedDataset(object):

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [13]:
# SFT_dataset 클래스를 사용해 훈련셋을 만들고 data collator 인스턴스 생성

train_dataset = SFT_dataset(data_path_1_SFT='KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

In [14]:
# 디코딩 함수 작성
def decode_tokens(tokenizer, input_ids, label_ids):
  """
  주어진 input_ids와 label_ids를 디코딩하여 원본 text를 출력하는 함수.
  label_ids에 포함된 -100 값은 디코딩에서 제외합니다.
  """
  # input_ids 디코딩
  # skip_special_tokens=True 옵션으로 </s>와 같은 특수 토큰을 제외하고 볼 수 있습니다.
  decoded_input = tokenizer.decode(input_ids, skip_special_tokens=True)

  # label_ids에서 -100을 제외한 토큰만 필터링
  filtered_label_ids = [token_id for token_id in label_ids if token_id != -100]

  # 필터링된 label_ids 디코딩
  decoded_label = tokenizer.decode(filtered_label_ids, skip_special_tokens=True)

  print("--- [디코딩 결과] ---")
  print(f"➡️ Input (전체 원본 문장):\n{decoded_input}\n")
  print(f"✅ Label (모델이 학습하는 정답 문장):\n{decoded_label}")

# 함수를 사용하여 첫 번째 데이터 확인
decode_tokens(
    tokenizer,
    train_dataset.input_ids[0],
    train_dataset.labels[0]
)

--- [디코딩 결과] ---
➡️ Input (전체 원본 문장):
### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.

✅ Label (모델이 학습하는 정답 문장):
'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.


In [15]:
# Training arguments를 사용해 trainer 클래스를 정의

# training_args = transformers.TrainingArguments(
#     output_dir="test",
#     overwrite_output_dir=True,
#     num_train_epochs=1,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     warmup_steps=5,
#     prediction_loss_only=True,
#     fp16 = True
#     )
training_args = transformers.TrainingArguments(
        output_dir="models/improved_sft_v2",
        overwrite_output_dir=True,
        num_train_epochs=3,  # 변경: 1 -> 3
        per_device_train_batch_size=4,  # 변경: 8 -> 4 (메모리 절약)
        gradient_accumulation_steps=4,  # 추가: 가상 배치 16
        learning_rate=2e-5,  # 변경: 기본값 -> 낮춤
        warmup_steps=100,  # 추가: 워밍업
        prediction_loss_only=True,
        fp16=True
        )
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)



In [16]:
# SFT 훈련을 진행

trainer.train()
model.save_pretrained('/content/drive/MyDrive/model_output/output_1_SFT')

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jinyongpark (jinyong_park) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,2.892400
1000,2.566100
1500,2.421800
2000,2.243200


###모델 성능 평가

In [17]:
# 문장 생성 능력을 확인하기 위해 빠르게 허깅페이스의 pipleline 클래스를 사용하여 generator 구현
generator = transformers.pipeline('text-generation', model='models/output_1_SFT', tokenizer=tokenizer)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

Device set to use cuda:0



### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 고기를 먹을 수 없습니다. 하지만 일반적으로 불고기는 쇠고기와 함께 먹는 음식 중 하나입니다. 따라서 고기를 먹을 수 있는 식당이나 레스토랑을 찾아보시는 것이 좋을 것 같습니다.墨書:\n\n불고기용 고기의 종류 및 가격은 다양합니다.

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨은 47대 부통령직을 수행하지 않았습니다.著述)은 "리처드 닉슨의 41대 부통령직 수행 년도는 1947년입니다.著述)는 리처드 닉슨이 51대 부통령직을 수행하는 동안 그의 지지자들이 그를 지지하지 않았기 때문에 정확한 답변을 제공하지 못합니다.著述은 리처드

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 시카고에 대한 정보를 가지고 있지 않습니다. 하지만 일반적으로 시카고 오헤어 공항은 미국 캘리포니아주 로스앤젤레스(LA) 지역에 위치해 있습니다.高速機械라고도 불립니다.高速器械라고도 부릅니다.膏速機械이라고도 부릅니다.

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 미세먼지에 대한 정보를 알 수 없습니다. 하지만 보통 미세먼지는 호흡기 질환, 심혈관 질환, 당뇨병 등 다양한 원인에 의해 발생할 수 있습니다. 따라서 외출 시 마스크를 착용하고, 실내에서 충분한 수면을 취하는 것이 좋습니다. 또한, 미세먼지가 심한 날


저는 인공지능 어시스턴트이기 때문에 "입력"에 대한 정보를 가지고 있지 않습니다

라는 프롬프트가 존재하는듯함

###Step 2. RM 학습
Ranknig셋으로 준비된 데이터를

pairwise형식으로 전환 후 학습시도



In [18]:
# RM 학습에 필요한 클래스 임포트
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer.strategies import NaiveStrategy
from chatgpt.trainer.rm import RewardModelTrainer

from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

import torch.nn as nn
import random

# SFT와 동일한 토크나이저 설정 사용
rm_tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

In [19]:
class GPTRM_custom(RewardModel):
    """
    GPT-2를 기반으로 하는 Custom Reward Model.
    입력된 텍스트의 좋고 나쁨을 판단하여 단일 점수(reward)를 출력합니다.
    """
    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:

        if pretrained is not None:
            # 사전 학습된 GPT2 모델을 불러옵니다.
            model = GPT2Model.from_pretrained(pretrained)
            # Special token이 추가된 토크나이저에 맞게 임베딩 크기를 조정합니다.
            model.resize_token_embeddings(len(tokenizer))
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())

        if checkpoint:
            model.gradient_checkpointing_enable()

        # 모델의 마지막 hidden state를 입력으로 받아 단일 점수를 출력하는 value_head를 정의합니다.
        value_head = nn.Linear(model.config.n_embd, 1)

        # 부모 클래스인 RewardModel을 초기화합니다.
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained

    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

# NaiveStrategy 컨텍스트 내에서 RM 모델을 초기화합니다.
with NaiveStrategy().model_init_context():
    rm_model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=rm_tokenizer).cuda()

In [20]:
# 준비된 RM 데이터셋 로드
with open('/content/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

# 랭킹 정보를 바탕으로 pairwise(chosen, rejected) 쌍 만들기
total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    # 3개의 답변 중 2개를 뽑는 모든 조합(3가지)에 대해 쌍을 생성
    # completion_0 vs completion_1
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    # completion_0 vs completion_2
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    # completion_1 vs completion_2
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)

    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


In [21]:
# 데이터 셔플 및 분할
random.seed(230319)
random.shuffle(total_data_ranking2chosen)

train_data = total_data_ranking2chosen[:1000]
eval_data = total_data_ranking2chosen[1000:1200]

print(f"Train data size: {len(train_data)}")
print(f"Eval data size: {len(eval_data)}")

# RewardDataset 객체 생성
train_dataset = RewardDataset(train_data, rm_tokenizer, 512)
eval_dataset = RewardDataset(eval_data, rm_tokenizer, 512)

# 데이터 샘플 확인
idx = 1
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

Train data size: 1000
Eval data size: 200


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

######################################################################
## prompt ##
흑고래의 무게는 어느 정도야
######################################################################
## chosen ##
흑고래의 평균 몸무게는 약 25~40톤 정도이지만, 최대 몸무게는 50톤 이상에 이를 수 있습니다.
######################################################################
## rejected ##
흑고래의 무게는 매우 다양하게 달라집니다. 약 200kg에서 10톤까지 달라질 수 있습니다.


In [22]:
# RM 트레이너 설정
rm_trainer = RewardModelTrainer(model=rm_model,
                             strategy=NaiveStrategy(),
                             optim=torch.optim.Adam(rm_model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=4,
                             max_epochs=1)

# RM 학습 시작
rm_trainer.fit(use_lora=0)

# 학습된 RM 모델 저장
rm_model.save_pretrained('/content/drive/MyDrive/model_output/output_2_RM')

Train epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Train step of epoch 0:   0%|          | 0/250 [00:00<?, ?it/s]

In [23]:
# RM 추론 함수 정의
def inference_RM(input_text):
    input_ids = rm_tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device()
    )
    output = rm_model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f' % (input_text, output_reward))
    return output_reward

# 테스트 1: 부정적인 문장
input_text = '인공지능은 똥멍청이 입니다'
output_reward = inference_RM(input_text=input_text)

# 테스트 2: 긍정적이고 짧은 문장
input_text = '인공지능(AI)은 매우 유용합니다.'
output_reward = inference_RM(input_text=input_text)

# 테스트 3: 긍정적이고 상세한 문장
input_text = "인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다."
output_reward = inference_RM(input_text=input_text)

# GPU 메모리 정리
torch.cuda.empty_cache()

input: 인공지능은 똥멍청이 입니다
reward score: -1.3
input: 인공지능(AI)은 매우 유용합니다.
reward score: -1.1
input: 인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다.
reward score: -1.4


###Step 4. PPO 적용

SFT, RM모델을 이용해 PPO를 수행


In [24]:
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer

from copy import deepcopy

In [26]:
# NaiveStrategy 컨텍스트 내에서 PPO 학습에 필요한 모든 모델을 준비합니다.
with NaiveStrategy().model_init_context():
    # Actor: SFT 모델을 불러옵니다.
    actor = GPTActor(pretrained='/content/drive/MyDrive/model_output/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())

    # Critic: RM 모델을 불러옵니다.
    critic = GPTCritic(pretrained='/content/drive/MyDrive/model_output/output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    # Tokenizer: 이전과 동일한 설정을 사용합니다.
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right",
        model_max_length=512
    )

    # Initial Model: SFT 모델을 복사하여 KL 페널티 계산에 사용합니다.
    initial_model = deepcopy(actor)

    # Reward Model: Critic 모델을 기반으로 보상 계산에 사용합니다.
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

# Actor와 Critic을 위한 옵티마이저를 설정합니다.
actor_optim = torch.optim.Adam(actor.parameters(), lr=5e-6)
critic_optim = torch.optim.Adam(critic.parameters(), lr=5e-6)

# Strategy를 통해 모델과 옵티마이저를 래핑합니다.
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [27]:
# PPO 학습을 위한 프롬프트 데이터 로드
with open('//content/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

# PPO Trainer 내부에서 사용할 토크나이저 함수 정의
def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

print(f"PPO 학습에 사용될 프롬프트 개수: {len(list_prompt)}")

PPO 학습에 사용될 프롬프트 개수: 12000


In [28]:
# PPOTrainer 초기화
trainer = PPOTrainer(NaiveStrategy(),
                     actor,           #학습이 진행될 모델
                     critic,          #RM
                     reward_model,    #RM
                     initial_model,   #SFT적용모델
                     actor_optim,
                     critic_optim,
                     max_epochs=1,
                     train_batch_size=8,
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

# PPO 학습 시작
trainer.fit(list_prompt,
            num_episodes=10,
            max_timesteps=3,
            update_timesteps=3)

# 최종 PPO 모델 저장
actor.model.save_pretrained('/content/drive/MyDrive/model_output/output_3_PPO')

Episode [1/10]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [1/1]:   0%|          | 0/3 [00:00<?, ?it/s]

Episode [2/10]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [1/1]:   0%|          | 0/3 [00:00<?, ?it/s]

Episode [3/10]:   0%|          | 0/3 [00:00<?, ?it/s]

...

Train epoch [1/1]:   0%|          | 0/3 [00:00<?, ?it/s]

Episode [9/10]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [1/1]:   0%|          | 0/3 [00:00<?, ?it/s]

Episode [10/10]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [1/1]:   0%|          | 0/3 [00:00<?, ?it/s]

In [29]:
# 최종 PPO 모델로 답변 생성
def generation(input_text, model, tokenizer):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())

    # PPO 모델은 Actor 클래스로 래핑되어 있으므로, 내부 모델을 직접 사용
    outputs = model.model.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)

    output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    print(output_text)
    return output_text

# 테스트용 프롬프트 리스트
list_prompt = [
    '불고기용 고기 한우에요?',
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

# SFT/PPO 모델에 맞는 프롬프트 형식으로 변환
PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}
list_formatted_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

print("--- PPO Model Generation Results ---")
for input_text in list_formatted_prompt:
    generation(input_text, actor, tokenizer)
    print("-" * 30)

--- PPO Model Generation Results ---
### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 챗봇이므로, 질문에 대한 정확성과 정확한 답변을 제공할 수 없습니다. 하지만, 쇠고기 판매점에서 직접 고기를 구매할 수 있는지 확인해보시는 것이 좋습니다.
------------------------------
### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨은 41대 부통령직을 수행했습니다.
------------------------------
### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어

### Response(응답):'시카고 오헤어 국제공항은 시카고에서 차로 약 1시간 정도 거리에 위치해 있습니다.
------------------------------
### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'저는 인공지능 챗봇이기 때문에 답변을 해드릴 수 없습니다. 하지만, 미세먼지 상황을 잘 살펴보고 대처하는 방법을 찾아보시는 것을 권장드립니다.
------------------------------


#실험 결과 확인
pipeline을 이용해 출력을 생성

In [62]:
from transformers import pipeline, AutoTokenizer
import torch, re

generators = {
    name: pipeline(
        "text-generation",
        model=path,
        tokenizer=tokenizer,
        device=device
    )
    for name, path in MODEL_PATHS.items()
}
print("ready:", list(generators.keys()))

#프롬프트 템플릿 + 종료 처리
PROMPT_DICT = {
    "prompt_input": "### Instruction(명령어):\n{prompt}\n\n### Response(응답):",
}
def build_prompt(p: str) -> str:
    return PROMPT_DICT["prompt_input"].format_map({"prompt": p})

# 종료 규칙: 응답 이후 불필요한 텍스트를 잘라주는 간단 후처리
def postprocess(prompt_text: str, generated_text: str) -> str:
    # 1) 프롬프트 에코 제거
    body = generated_text[len(prompt_text):] if generated_text.startswith(prompt_text) else generated_text

    # 2) "다음 섹션"을 암시하는 마커가 보이면 거기서 자르기 (옵션)
    cut_points = [
        "\n\n###",             # 다음 섹션 마커
        "\n\nInstruction",     # 잘못 에코 시
        "\n\n명령어",           # 한글 에코 시
    ]
    for cp in cut_points:
        if cp in body:
            body = body.split(cp)[0]
            break

    # 3) 과도한 공백 정리
    body = body.strip()
    return body

# 종료 토큰: 개행을 EOS로 쓰고 싶다면 토크나이저에서 id 찾기
EOS_NL = tokenizer.encode("\n", add_special_tokens=False)[0]



Device set to use cuda
Device set to use cuda
Device set to use cuda


ready: ['base', 'sft', 'ppo']


In [63]:
GEN_ARGS_GREEDY = dict(
    do_sample=False,
    num_beams=4,
    max_new_tokens=96,
    no_repeat_ngram_size=4,
    repetition_penalty=1.2,
    eos_token_id=EOS_NL,                  # 응답 끝나면 개행에서 끊기
    pad_token_id=tokenizer.eos_token_id,
    early_stopping=True,
)


In [64]:
list_prompt = [
    "불고기용 고기 한우에요?",
    "리처드 닉슨이 43대 부통령직을 수행한 년도는?",
    "시카고 오헤어 국제공항은 어디에 있어?",
    "오늘 미세먼지 어때?"
]
batched_inputs = [build_prompt(p) for p in list_prompt]

def run_and_show(generators, batched_inputs, gen_args):
    # 각 모델을 순회하며 배치 생성
    all_outputs = {name: gen(batched_inputs, **gen_args) for name, gen in generators.items()}

    # 프롬프트별로 묶어서 출력
    for i, raw_prompt in enumerate(list_prompt):
        full_prompt = batched_inputs[i]
        print("="*100)
        print(f"PROMPT {i+1}: {raw_prompt}")
        for name in ["base", "sft", "ppo"]:
            res_text = all_outputs[name][i][0]["generated_text"]
            ans = postprocess(full_prompt, res_text)
            print(f"\n[{name.upper()}]\n{ans}\n")

#  그리디 비교
run_and_show(generators, batched_inputs, GEN_ARGS_GREEDY)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


PROMPT 1: 불고기용 고기 한우에요?

[BASE]
#food #delicious #selfie #foodstagram #instafood #먹스타그램 #먹방 #먹짤 #위꼴 #데일리 #일상 #소통 #팔로우 #맞팔 #선팔 #인친 #인스타푸드 #좋아요
#f4f #l4


[SFT]
'죄송합니다, 저는 인공지능 어시스턴트이기 때문에 실제로 고기를 먹을 수 없습니다. 하지만 인터넷에서 검색해보시면 불고기용 고기를 판매하는 가게가 많이 있을 것 같습니다. 감사합니다.述, "불고기용 고기에 대한 자세한 정보를 알려주시면 더 정확한 답변을 드릴 수 있을 것 같습니다!", 'token': 222}述, "이러한 정보를 알려주셔서 감사합니다.


[PPO]
'저는 AI 어시스턴트이기 때문에 고기를 먹을 수 없습니다. 하지만 일반적으로 불고기용 고기는 쇠고기, 돼지고기, 소고기, 닭고기 등 다양한 부위를 사용합니다. 따라서 고기를 먹을 수 있는 식당이나 음식점을 찾아보시는 것이 좋을 것 같습니다.述:\n\n저는 인공지능 어시스턴트로써 고기를 먹을 수 없기 때문에 고기를 먹을 수는 없습니다. 하지만 인터넷에서 검색해보시면 맛있는 고기를 찾을 수 있을 것입니다.述:

PROMPT 2: 리처드 닉슨이 43대 부통령직을 수행한 년도는?

[BASE]
## # # #20190413 #미롱_식단 
아침 : #아침식단
공복시간 : #점심식단
#diet #dieter #다이어트그램 #오늘의식단 #식단일기
#다이어트레시피 #직장인다
#식단조절 #건강식단


[SFT]
'리처드 닉슨은 47대 부통령직을 수행했습니다.辛巳申巳申巳辛巳辛巳新巳辛巳神巳辛巳身巳神巳神巳新巳神巳身巳新巳身巳身巳辛巳신사辛巳身四神身四神神四神四神五神五神四神三神五神三神四神六神五神六神六神三神三神六神四神


[PPO]
'리처드 닉슨은 47대 부통령직을 수행했습니다. ector of the physical context of the scientific relationships. ectors of the capability of the 

베이스 모델에서 sft, PPO를 거칠수록 점점 답변이 깨끗해지는걸 볼 수 있다.


--------------------------------------
--------------------------------------

#Beam search 적용
--------------------------------------
--------------------------------------


In [67]:
# 개행 EOS는 이전에 만든 EOS_NL 사용(없으면 아래 줄로 구하세요)
# EOS_NL = tokenizer.encode("\n", add_special_tokens=False)[0]

GEN_ARGS_BEAM = dict(
    do_sample=True,          # 🔒 순수 빔서치 (랜덤성 X)
    num_beams=7,
    max_new_tokens=96,
    no_repeat_ngram_size=4,
    repetition_penalty=1.2,
    eos_token_id=EOS_NL,      # 응답 끝을 개행으로 유도
    pad_token_id=tokenizer.eos_token_id,
    early_stopping=True,
)

# 실행
run_and_show(generators, batched_inputs, GEN_ARGS_BEAM)

PROMPT 1: 불고기용 고기 한우에요?

[BASE]
#foodie #foodstagram #instafood #delicious #yammy #fff #f4f #l4l #follow #like4like #소통 #좋아요
#팔로우 #맞팔 #선팔 #일상 #데일리 #먹방 #먹스타그램


[SFT]
'죄송합니다, 저는 인공지능 어시스턴트이기 때문에 고기를 먹을 수 없습니다. 하지만 인터넷에서 검색해보시면 불고기용 고기의 종류를 쉽게 찾을 수 있을 것입니다. \n\n저는 AI 어시스턴트로써 고기를 먹을 수 없기 때문에 직접 고기를 먹을 수는 없습니다. 그러나 인터넷에서 검색해보면 불고기용 고기를 판매하는 가게가 많이 있으니 찾아보시는 것을 추천드립니다. 감사합니다.纏前前前前


[PPO]
'죄송합니다, 저는 인공지능 어시스턴트이기 때문에 고기를 먹을 수 없습니다. 하지만 인터넷에서 검색해보시는 것을 추천드립니다!高辛肉, 高辛肉,高辛肉 등 다양한 종류의 고기를 구매하시면 좋을 것 같습니다.高辛六, 高辛六,高辛六 등 다양한 종류의 고기가 있으니 맛있게 드시길 바랍니다.高辛三, 高辛三,高辛三

PROMPT 2: 리처드 닉슨이 43대 부통령직을 수행한 년도는?

[BASE]
## # # #2017프로젝트_에일린 #프로젝트ELIGHT 
프로젝트 ELIG HT RESERVED!!!!!
프로젝트 HT SHOW COMPANY!!! 
#2018프로젝트HT #SHOWCHOW


[SFT]
'리처드 닉슨은 47대 부통령직을 수행하지 않았습니다.著)은 리처드 닉슨이 41대 부통령직을 맡았던 시기에 대한 정보를 찾을 수 없습니다.著)는 리처드 닉슨이 44대 부통령직을 맡은 시기와 관련된 정보를 찾을 수 없기 때문입니다.著는 리처드 닉슨이 46대 부통령직을 맡았는지에 대한 추가 정보를 제공할 수 있습니다.著은 리처드 닉슨이 47대 부통령을 맡았던 시기와 관련된 정보를 제공할 수 없기 때문에 정확한 답변을 제공하기 어렵습니다.著는


[PPO]
'리처드 닉슨은 47대 부통령직을 수행하지 않았

Beam Search를 적용한 모델의 답변의 어휘가 기존에 비해 편해졌다.
적용하지 않은 출력에서는 기존에 주어진 단어들을 우선적으로 선택해 어투가 딱딱한 느낌이 든 반면
Beam Search를 적용한 출력에서는 좀 더 다양한 어휘를 보이며 자연스러운 출력을 보였다.

###EX 01. 추가 데이터 정제

주말사이에 시도해보겠습니다.